In [1]:
import os
from dotenv import load_dotenv
load_dotenv()# loading all the enviroment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key


'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023EAC43E7D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023EAD95BFD0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi,My name is rutuja")])

AIMessage(content="Hi Rutuja, it's nice to meet you! 👋\n\nWhat can I do for you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 17, 'total_tokens': 44, 'completion_time': 0.049090909, 'prompt_time': 0.000634317, 'queue_time': 0.081991663, 'total_time': 0.049725226}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e9ad6796-16dd-454d-99b4-52f4ced8d8c9-0', usage_metadata={'input_tokens': 17, 'output_tokens': 27, 'total_tokens': 44})

In [4]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi,My name is rutuja"),
    AIMessage(content="Hi Rutuja, it's nice to meet you!\n\nWhat can I do for you today? 😊  \n\n"),
    HumanMessage(content="what is my name?")
])

AIMessage(content='Your name is Rutuja.  You told me at the beginning of our conversation!  😊 \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 57, 'total_tokens': 92, 'completion_time': 0.063636364, 'prompt_time': 0.001685143, 'queue_time': 0.012065017, 'total_time': 0.065321507}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1bc7a3d6-3887-4877-941f-18d5d83bffc3-0', usage_metadata={'input_tokens': 57, 'output_tokens': 35, 'total_tokens': 92})

In [5]:
#Message history
'''We can use a Message History class to wrap our model and make it stateful. 
This will keep track of inputs and outputs of the model, and store them in some datastore. 
Future interactions will then load those messages and pass them into the chain as part of the input. 
'''

'We can use a Message History class to wrap our model and make it stateful. \nThis will keep track of inputs and outputs of the model, and store them in some datastore. \nFuture interactions will then load those messages and pass them into the chain as part of the input. \n'

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="hi my name is rutuja")],
    config=config
)

In [9]:
response.content

"Hi Rutuja, it's nice to meet you!👋 \n\nWhat can I do for you today? 😄 \n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config
)

AIMessage(content='Your name is Rutuja! 😊  I remember that you told me earlier.  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 58, 'total_tokens': 90, 'completion_time': 0.058181818, 'prompt_time': 0.001701854, 'queue_time': 0.011567213, 'total_time': 0.059883672}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-988cf809-5dd6-4e25-920e-68225df46c54-0', usage_metadata={'input_tokens': 58, 'output_tokens': 32, 'total_tokens': 90})

In [11]:
#change the config -->means change the session_id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="my name is rutuja")],
    config=config1
)
response.content

"Hello, Rutuja! It's nice to meet you. 😊\n\nIs there anything I can help you with today?\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
)
response.content

'Your name is Rutuja.  \n\nYou told me a little while ago! 😊  Do you want to talk about something else?\n'

In [14]:
#prompt template
'''Prompt Templates help to tum raw user information into a format that the LLM can work with. 
In this case, the raw user input is just a message, which we are passing to the LLM. 
Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions 
(but still taking messages as input). 
Next, we'll add in more input besides just the messages.'''

"Prompt Templates help to tum raw user information into a format that the LLM can work with. \nIn this case, the raw user input is just a message, which we are passing to the LLM. \nLet's now make that a bit more complicated. First, let's add in a system message with some custom instructions \n(but still taking messages as input). \nNext, we'll add in more input besides just\xa0the\xa0messages."

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi my  name is rutuja")]})

AIMessage(content="Hi Rutuja, it's nice to meet you!  \n\nI'm happy to help with any questions you have.  Just ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 33, 'total_tokens': 71, 'completion_time': 0.069090909, 'prompt_time': 0.001171356, 'queue_time': 0.02107823, 'total_time': 0.070262265}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-183d4cf4-031a-4bce-94dd-60a20168b238-0', usage_metadata={'input_tokens': 33, 'output_tokens': 38, 'total_tokens': 71})

In [17]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is rutuja")],
    config=config
)
response

AIMessage(content="Hello Rutuja! It's nice to meet you. \n\nI'm ready to answer your questions to the best of my ability. What can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 32, 'total_tokens': 76, 'completion_time': 0.08, 'prompt_time': 0.000309078, 'queue_time': 0.0132857, 'total_time': 0.080309078}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e3b4a64e-1a11-4bbd-bf8c-789a8ef46020-0', usage_metadata={'input_tokens': 32, 'output_tokens': 44, 'total_tokens': 76})

In [28]:
#add more complexity
prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a helpful assistant. answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain=prompt|model

In [29]:
response=chain.invoke({"messages":[HumanMessage(content="hi my name is rutuja")],"language":"hindi"})
response.content

'नमस्ते रूतुजा! 😊 \n\nमुझे बहुत खुशी है कि आपने मुझसे बात करना चुना। आप मुझसे जो भी पूछना चाहें, मैं अपनी पूरी कोशिश करूँगा कि आपको सही जवाब दूं। \n\n'

Lets now wrap this more complicated chain in a message history class.This time, because there are multiple keys in the input,we need to specify the correct key to use to save the chat history.

In [30]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [31]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="hi  i am rutuja")],"language":"hindi"},
    config=config
)
response.content

'नमस्ते रूतुजा! 😊\n\nआपका नाम रूतुजा है, मैंने ठीक से सुना। \n\nआप क्या पूछना चाहेंगे? \n'

In [32]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="what is my name?")],"language":"hindi"},
    config=config
)
response.content

'आपका नाम रूतुजा है! 😊 \n\nक्या आप कोई और सवाल पूछना चाहेंगे?  \n'

Managing the conversation history


One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in. 

'trim messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [36]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages=[
SystemMessage(content="you're a good assistant"),
HumanMessage(content="hi! I'm bob"),
AIMessage(content="hi!"),
HumanMessage(content="I like vanilla ice cream"),
AIMessage(content="nice"),
HumanMessage(content="whats 2 + 2"),
AIMessage(content="4"),
HumanMessage(content="thanks"), AIMessage(content="no problem!"),
HumanMessage(content="having fun?"),
AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [42]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt|model
)
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what ice cream do i like")],
    "language":"English"
    }
)
response.content
#context is not their because of trimmer so we wont get correct answer

"As an AI, I don't have access to your personal information, including your taste preferences like ice cream flavors.  \n\nWhat's your favorite ice cream flavor? 😊  \n"

In [41]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what math problem did i ask")],
    "language":"English"
    }
)
response.content


'You asked "What is 2 + 2?".  😊 \n\n\n\nLet me know if you want to try another one!\n'

In [43]:
#lets wrap this in the message history
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [47]:
response=with_message_history.invoke(
    {
    "messages":messages + [HumanMessage(content="what is my name")],
    "language":"English",
    },
    config=config
)
response.content


"As an AI, I don't have access to past conversations or any personal information about you, including your name. If you'd like to tell me your name, I'd be happy to know! 😊  \n\n\n"